<a href="https://colab.research.google.com/github/vitormanfredini/clipe-sobre-uma-mesa-google-collab/blob/main/clipe_Vitor_Sobre_uma_mesa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<h1>Código usado para criar o <a href="https://www.youtube.com/watch?v=RF8BjlrvNx4&ab_channel=V%C3%ADtorManfredini">clipe de uma música</a> usando IA de super resolução + script de keyframes</h1>

In [ ]:
#@title #Monta Google Drive

import os
from google.colab import drive
if not os.path.isdir('/content/drive'):
  drive.mount('/content/drive')
if not os.path.isdir('/content/mydrive'):
  os.symlink('/content/drive/My Drive', '/content/mydrive')

In [ ]:
#@title #Instala dependências

from google.colab import output
import requests, cv2

!pip -q install import-ipynb typer rich
!apt-get update
!apt -qq install imagemagick

# clone repos
!mkdir -p /content/mydrive/clipe/esrgan/
%cd /content/mydrive/clipe/esrgan/
!git clone https://github.com/olaviinha/ESRGAN.git Colab-ESRGAN

%cd "/content/mydrive/clipe/esrgan/Colab-ESRGAN"

dir_tmp = '/content/mydrive/clipe/esrgan/Colab-ESRGAN/tmp/'
dir_mask = '/content/mydrive/clipe/esrgan/Colab-ESRGAN/tmp/mask/'
dir_input = '/content/mydrive/clipe/esrgan/Colab-ESRGAN/input/'
dir_dejpeg = '/content/mydrive/clipe/esrgan/Colab-ESRGAN/dejpeg/'
dir_upscaled = '/content/mydrive/clipe/esrgan/Colab-ESRGAN/upscaled/'
dir_output = '/content/mydrive/clipe/esrgan/Colab-ESRGAN/output/'
dir_models = '/content/mydrive/clipe/esrgan/Colab-ESRGAN/models/'
finalOutputDir = "/content/mydrive/clipe/output"
tmpDir = "/content/mydrive/clipe/output-tmp"

!mkdir -p dir_tmp
!mkdir -p dir_mask
!mkdir -p dir_input
!mkdir -p dir_dejpeg
!mkdir -p dir_upscaled
!mkdir -p dir_output
!mkdir -p dir_models
!mkdir -p $finalOutputDir
!mkdir -p $tmpDir

# download models
if False == os.path.isfile(dir_models+'RRDB_ESRGAN_x4.pth'):
  !wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1MJFgqXJrMkPdKtiuy7C6xfsU1QIbXEb-' -O models/RRDB_ESRGAN_x4.pth
if False == os.path.isfile(dir_models+'RRDB_PSNR_x4.pth'):
  !wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1mSJ6Z40weL-dnPvi390xDd3uZBCFMeqr' -O models/RRDB_PSNR_x4.pth
if False == os.path.isfile(dir_models+'4x_FatalAnime_500000_G.pth'):
  !wget --no-check-certificate "https://de-next.owncube.com/index.php/s/x99pKzS7TNaErrC/download" -O models/4x_FatalAnime_500000_G.pth
if False == os.path.isfile(dir_models+'4x_FatalPixels.pth'):
  !wget --no-check-certificate "https://de-next.owncube.com/index.php/s/mDGmi7NgdyyQRXL/download?path=%2F&files=4x_FatalPixels_340000_G.pth&downloadStartSecret=r4q3aw60ijm" -O models/4x_FatalPixels.pth
if False == os.path.isfile(dir_models+'1x_DeJpeg_Fatality_PlusULTRA.pth'):
  !wget --no-check-certificate "https://de-next.owncube.com/index.php/s/w82HLrLWmWi4SQ5/download" -O models/1x_DeJpeg_Fatality_PlusULTRA.pth
if False == os.path.isfile(dir_models+'4x_NMKD-YandereNeo-Lite_320k.pth'):
  !gdown --id 14lA-Ks5quxheNyVeXRvzeoSAOm6ISDHn -O models/4x_NMKD-YandereNeo-Lite_320k.pth

output.clear()
print('Setup finished.')

In [ ]:
#@title #Define funções
#@markdown 
import hashlib
import subprocess

def cleanAllDirs():
  for dir in [dir_input,dir_mask,dir_dejpeg,dir_upscaled]:
    if os.path.isdir(dir):
      for file in os.listdir(dir):
        filename = dir+file 
        !echo '' > $filename && rm $filename

def getModelByIndex(index):
  return [
      'RRDB_ESRGAN_x4.pth',
      '4x-UltraSharp.pth',
      '4x_Faces_04_N_180000_G.pth',
      '4x-Fabric.pth',
      '4xBox.pth',
      '4x_Valar_v1.pth'
      ][index]

def keyFramesToFrames(keyframes):
  
  totalTimeSeconds = keyframes[-1][0]
  fps = 30
  oneFrameInSeconds = 1.0 / fps

  frames = []
  for f in range(int(totalTimeSeconds * fps)):
    currentTime = f * oneFrameInSeconds

    for idx, keyframe in enumerate(reversed(keyframes)):
      if keyframe[0] <= currentTime:
        currentKeyFrame = keyframe
        break

    frame = currentKeyFrame.copy()
    if f > 0:
      for i in range(len(frame)):
        if frame[i] == None:
          frame[i] = frameAnterior[i]
    
    # adiciona sem o parâmetro de tempo
    frames.append(frame[1:])
    frameAnterior = frame.copy()

  return frames

# hue vai de 100 a 300 (e é cíclico)
def getCurrentHue(frames,currentFrame):
  # se não é pra fazer hue rotation
  if frames[currentFrame][7] != True:
    return 100
  # se é pra fazer, procura primeiro frame para calcular a partir dele
  for i in range(currentFrame-1):
    if frames[currentFrame-i][7] == True and frames[currentFrame-i-1][7] != True:
      return 100 + (i % 200)
  return 100

def generateTextImage(text,frameLetraColor,size,x,y,tmpDir,width,height,hue):
  
  # gera nome do arquivo fazendo hash dos parâmetros
  all = text+"-"+frameLetraColor+"-"+str(hue)+"-"+str(size)+"-"+str(x)+"-"+str(y)+"-"+str(width)+"-"+str(height)
  m = hashlib.md5()
  m.update(all.encode('utf-8'))
  hash = m.hexdigest()
  image = tmpDir+"/"+hash+".png"

  if os.path.isfile(image):
    return image

  x = int(width * x)
  y = int(height * y)
  # correção de x e y para tentar centraliza o texto
  x -= int(len(text) * (size * 0.25))
  y += size * 0.25

  fontFilename = '/content/mydrive/font.otf'
  strWidthHeight = str(width)+"x"+str(height)
  strDraw = '"fill '+frameLetraColor+' text '+str(x)+','+str(y)+' \''+text+'\'"'
  if os.path.isfile(fontFilename):
    !convert -size {strWidthHeight} xc:none -font {fontFilename} -pointsize {size} -draw {strDraw} {image}
  else:
    !convert -size {strWidthHeight} xc:none -pointsize {size} -draw {strDraw} {image}
  
  if hue != 100:
    !convert {image} -modulate 100,100,{hue} {image}

  assert os.path.isfile(image)

  return image

def getStepToResume(dir):
  cmd = 'ls '+dir+' | wc -l'
  output = subprocess.check_output(cmd, shell=True);
  return int(output.decode('utf-8').strip())

def debugFramesAndExit(arrFrames,numberOfFrames):
  for i in range(numberOfFrames):
    print(arrFrames[i])
  assert False

In [ ]:
#@title #Script do clipe

#@markdown <p>Para cada keyframe temos as seguintes configurações:</p>
#@markdown <ul>
#@markdown <li>tempo em segundos (float)</li>
#@markdown <li>zoom (float de 0 a 1)</li>
#@markdown <li>modelo de IA usado para fazer upscale (int de 0 a 5)</li>
#@markdown <li>texto para adicionar no frame (string)</li>
#@markdown <li>cor do texto em hex (ex: #ff00ff)</li>
#@markdown <li>tamanho do texto em pontos (int)</li>
#@markdown <li>posição horizontal do texto (float de 0 a 1)</li>
#@markdown <li>posição vertical do texto (float de 0 a 1)</li>
#@markdown <li>hue rotation (True ou False)</li>
#@markdown </ul>
#@markdown <i>Obs: use None em qualquer um dos campos (exceto o tempo) para repetir o valor anterior.</i>


arrKeyFrames = []
arrKeyFrames.append([0.0, 0.15, 1, '', '#ffffff', 120, 0.5, 0.5, False]);
arrKeyFrames.append([0.925338, None, 0, None, None, None, None, None, None]);
arrKeyFrames.append([2.172807, None, 5, None, None, None, None, None, None]);
arrKeyFrames.append([2.659162, None, 0, None, None, None, None, None, None]);
arrKeyFrames.append([3.470806, None, 1, None, None, None, None, None, None]);
arrKeyFrames.append([4.392986, None, 0, None, None, None, None, None, None]);
arrKeyFrames.append([5.432017, None, 5, None, None, None, None, None, None]);
arrKeyFrames.append([5.757306, None, 1, None, None, None, None, None, None]);
arrKeyFrames.append([6.088912, None, 0, None, None, None, None, None, None]);
arrKeyFrames.append([6.954245, None, 1, None, None, None, None, None, None]);
arrKeyFrames.append([7.873266, None, 0, None, None, None, None, None, None]);
arrKeyFrames.append([9.133368, None, 5, None, None, None, None, None, None]);
arrKeyFrames.append([9.616565, None, 0, None, None, None, None, None, None]);
arrKeyFrames.append([10.462949, None, 4, None, None, None, None, None, None]);
arrKeyFrames.append([11.34723, None, 0, None, None, None, None, None, None]);
arrKeyFrames.append([12.165191, None, 3, None, None, None, None, None, None]);
arrKeyFrames.append([12.831561, None, None, 'aqui', None, None, 0.5, 0.42, None]);
arrKeyFrames.append([13.570567, None, None, 'estou', None, None, 0.5, 0.58, None]);
arrKeyFrames.append([13.902173, 0.3, 2, None, '#32C999', 80, None, None, None]);
arrKeyFrames.append([14.789613, None, None, '', None, None, 0.5, 0.5, None]);
arrKeyFrames.append([17.395086, None, 5, None, None, None, None, None, None]);
arrKeyFrames.append([18.247787, None, None, 'sobre uma mesa', '#a7bb23', None, 0.5, 0.5, None]);
arrKeyFrames.append([19.132068, None, 4, None, None, None, None, None, None]);
arrKeyFrames.append([20.123727, None, None, 'tiro a roupa', '#af0053', None, 0.5, 0.5, None]);
arrKeyFrames.append([20.809245, None, 1, None, None, None, None, None, None]);
arrKeyFrames.append([23.067122, None, None, '', None, None, 0.5, 0.5, None]);
arrKeyFrames.append([23.228188, None, None, 'mas sigo', None, None, 0.5, 0.42, None]);
arrKeyFrames.append([23.90719, None, None, 'coberto', None, None, 0.5, 0.58, None]);
arrKeyFrames.append([24.34386, None, 1, None, None, None, None, None, None]);
arrKeyFrames.append([26.222113, None, None, '', None, None, 0.5, 0.5, None]);
arrKeyFrames.append([27.50748, None, None, 'do homem', '#94b5eb', None, 0.5, 0.42, None]);
arrKeyFrames.append([29.099187, None, None, 'que ainda', '#94b5eb', None, 0.5, 0.58, None]);
arrKeyFrames.append([30.829853, None, None, 'não sou', '#5069d9', None, 0.5, 0.5, None]);
arrKeyFrames.append([33.1732, None, None, '', None, None, 0.5, 0.5, None]);
arrKeyFrames.append([34.259603, None, None, 'do homem', '#e6b120', None, 0.5, 0.58, None]);
arrKeyFrames.append([36.015534, None, None, 'que não', '#c2c221', None, 0.5, 0.42, None]);
arrKeyFrames.append([37.139836, None, None, 'deixei de ser', '#729917', None, 0.5, 0.5, None]);
arrKeyFrames.append([40.013751, None, None, '', None, None, 0.5, 0.5, None]);
arrKeyFrames.append([40.601167, None, None, 'das dificuldades', '#841f94', None, 0.5, 0.5, None]);
arrKeyFrames.append([43.92354, None, None, '', None, None, 0.5, 0.5, None]);
arrKeyFrames.append([44.72571, None, None, 'que tenho', '#bbbbbb', None, 0.5, 0.5, None]);
arrKeyFrames.append([46.746926, None, None, '', None, None, 0.5, 0.5, None]);
arrKeyFrames.append([49.799517, None, None, 'e das', '#ffffff', 110, 0.5, 0.4, None]);
arrKeyFrames.append([50.300000, None, None, 'que me tem', '#bf1f7f', 135, 0.5, 0.5, None]);
arrKeyFrames.append([52.164496, None, 4, '', None, None, 0.5, 0.5, None]);
arrKeyFrames.append([53.918951, None, 3, None, None, None, None, None, None]);
arrKeyFrames.append([55.666979, 0.55, 2, 'cascas', '#eb8334', 120, 0.5, 0.5, True]);
arrKeyFrames.append([58.478605, None, None, 'não servem mais', None, None, 0.5, 0.5, None]);
arrKeyFrames.append([61.756157, None, None, '', None, None, 0.5, 0.5, None]);
arrKeyFrames.append([62.141752, None, None, 'precisei', None, None, 0.5, 0.5, None]);
arrKeyFrames.append([63.018979, None, None, 'pra proteger', None, None, 0.5, 0.5, None]);
arrKeyFrames.append([64.876259, None, None, '', None, None, 0.5, 0.5, None]);
arrKeyFrames.append([65.21044, None, None, 'tudo que foi', None, None, 0.5, 0.5, None]);
arrKeyFrames.append([67.51758, None, None, '', None, None, 0.5, 0.5, None]);
arrKeyFrames.append([69.104944, None, None, 'sentido', None, None, 0.5, 0.5, None]);
arrKeyFrames.append([69.300000, None, 4, None, None, None, 0.5, 0.5, None]);
arrKeyFrames.append([71.733412, None, None, 'e não foi cuidado', None, None, 0.5, 0.5, None]);
arrKeyFrames.append([75.216615, None, None, '', None, None, 0.5, 0.5, False]);
arrKeyFrames.append([76.090629, None, None, 'mas pra ser', None, None, 0.5, 0.36, None]);
arrKeyFrames.append([76.790629, None, None, 'ouvido', None, None, 0.5, 0.42, None]);
arrKeyFrames.append([77.812951, None, None, 'e escutar', None, None, 0.5, 0.58, None]);
arrKeyFrames.append([79.548126, None, None, 'tem que deixar', '#7a4ab5', 100, 0.5, 0.5, None]);
arrKeyFrames.append([79.97228, None, None, None, None, None, None, None, None]);
arrKeyFrames.append([81.514657, None, None, 'perto chegar', '#3dd193', 120, 0.5, 0.5, True]);
arrKeyFrames.append([83.429776, 0.15, 5, '', None, 80, None, None, False]);
arrKeyFrames.append([86.961178, None, 3, None, None, None, None, None, None]);
arrKeyFrames.append([90.405822, None, 4, None, None, None, None, None, None]);
arrKeyFrames.append([93.908304, None, 2, None, None, None, None, None, None]);
arrKeyFrames.append([97.381867, 0.2, 1, None, '#71DEDA', 80, None, None, None]);
arrKeyFrames.append([98.866405, None, None, 'ouvir', '#fff173', 93, 0.4, 0.6, None]);
arrKeyFrames.append([99.492996, None, None, 'sentir', '#fff173', None, 0.5, 0.5, None]);
arrKeyFrames.append([100.42485, None, None, 'calar', '#fff173', None, 0.6, 0.4, None]);
arrKeyFrames.append([101.845122, None, None, '', None, None, 0.5, 0.5, None]);
arrKeyFrames.append([102.352822, None, None, 'e esperar', '#ffa473', 80, 0.52, 0.66, None]);
arrKeyFrames.append([104.07193, None, None, 'e escutar', '#ffa473', None, 0.58, 0.58, None]);
arrKeyFrames.append([106.719678, None, None, '', None, None, 0.5, 0.5, None]);
arrKeyFrames.append([107.523, None, None, 'uô', '#7affad', 120, 0.4, 0.4, None]);
arrKeyFrames.append([107.828262, None, None, 'ô', '#7affad', None, 0.5, 0.45, None]);
arrKeyFrames.append([108.119065, None, None, 'ô', '#7affad', None, 0.6, 0.66, True]);
arrKeyFrames.append([111.462489, None, None, '', None, None, 0.5, 0.5, False]);
arrKeyFrames.append([113.004867, None, None, 'ô', '#f194ff', None, 0.5, 0.5, True]);
arrKeyFrames.append([113.93672, None, None, 'ô', None, None, 0.7, 0.3, None]);
arrKeyFrames.append([114.810734, None, None, 'ô', None, None, 0.4, 0.4, None]);
arrKeyFrames.append([115.729734, None, None, 'ô', None, None, 0.5, 0.5, None]);
arrKeyFrames.append([118.98158, None, None, '', None, None, 0.5, 0.5, False]);
arrKeyFrames.append([125.215356, 0.15, 1, None, '#ffffff', 120, None, None, None]);
arrKeyFrames.append([126.140694, None, 0, None, None, None, None, None, None]);
arrKeyFrames.append([127.388163, None, 5, None, None, None, None, None, None]);
arrKeyFrames.append([127.874518, None, 0, None, None, None, None, None, None]);
arrKeyFrames.append([128.686163, None, 1, None, None, None, None, None, None]);
arrKeyFrames.append([129.608342, None, 0, None, None, None, None, None, None]);
arrKeyFrames.append([130.647373, None, 5, None, None, None, None, None, None]);
arrKeyFrames.append([130.972663, None, 1, None, None, None, None, None, None]);
arrKeyFrames.append([131.304268, None, 0, None, None, None, None, None, None]);
arrKeyFrames.append([132.169601, None, 1, None, None, None, None, None, None]);
arrKeyFrames.append([133.088623, None, 0, None, None, None, None, None, None]);
arrKeyFrames.append([134.348724, None, 5, None, None, None, None, None, None]);
arrKeyFrames.append([134.831921, None, 0, None, None, None, None, None, None]);
arrKeyFrames.append([135.678305, None, 5, None, None, None, None, None, None]);
arrKeyFrames.append([136.562587, None, 0, None, None, None, None, None, None]);
arrKeyFrames.append([137.380548, None, 3, None, None, None, None, None, None]);
arrKeyFrames.append([139.173874, 0.3 , 5, None, None, None, None, None, None]);
arrKeyFrames.append([142.618517, None, 4, None, None, None, None, None, None]);
arrKeyFrames.append([146.069587, None, 3, None, None, None, None, None, None]);
arrKeyFrames.append([149.578496, None, 1, None, None, None, None, None, None]);
arrKeyFrames.append([151.377937, None, 0, None, None, None, None, None, None]);
arrKeyFrames.append([153.050453, None, 5, None, None, None, None, None, None]);
arrKeyFrames.append([156.512769, None, 4, None, None, None, None, None, None]);
arrKeyFrames.append([159.995972, None, 3, None, None, None, None, None, None]);
arrKeyFrames.append([163.498454, None, 4, None, None, None, None, None, None]);
arrKeyFrames.append([165.240056, None, 3, None, None, None, None, None, None]);
arrKeyFrames.append([166.962377, None, 1, None, None, None, None, None, None]);
arrKeyFrames.append([170.432727, None, None, None, None, None, None, None, None]);
arrKeyFrames.append([173.909503, None, 5, None, None, None, None, None, None]);
arrKeyFrames.append([177.405559, None, None, None, None, None, None, None, None]);
arrKeyFrames.append([180.895189, None, 4, None, None, None, None, None, None]);
arrKeyFrames.append([184.371965, None, 5, None, None, None, None, None, None]);
arrKeyFrames.append([187.842314, None, 4, None, None, None, None, None, None]);
arrKeyFrames.append([191.325517, None, 2, None, None, None, None, None, None]);
arrKeyFrames.append([193.478419, 0.0, 0, None, None, None, None, None, None]);
arrKeyFrames.append([194.776587, 0.15, 4, None, None, None, None, None, None]);
arrKeyFrames.append([198.24051, None, 3, None, None, None, None, None, None]);
arrKeyFrames.append([201.100335, None, 1, None, None, None, None, None, None]);
arrKeyFrames.append([201.434517, None, 0, None, None, None, None, None, None]);
arrKeyFrames.append([201.749419, None, 5, None, None, None, None, None, None]);
arrKeyFrames.append([205.219769, None, 2, None, None, None, None, None, None]);
arrKeyFrames.append([212.02551, None, None, None, None, None, None, None, None]);
# o último keyframe indica o fim

In [ ]:
#@title #Rodar
from google.colab import output
import cv2

%cd "/content/mydrive/clipe/esrgan/Colab-ESRGAN"

arrFrames = keyFramesToFrames(arrKeyFrames)
# debugFramesAndExit(arrFrames,1000)

initialStep = getStepToResume(finalOutputDir)
if initialStep == 0:
  inputImage = tmpDir+'/randomInput.png'
  !convert -size 1080x1920 xc: +noise Random {inputImage} # cria imagem 100% aleatória para input
else:
  # continua deste arquivo
  inputImage = finalOutputDir+"/"+ ('%06d' % (initialStep-1)) + ".png"

assert os.path.isfile(inputImage)

im = cv2.imread(inputImage)
inputHeight, inputWidth, inputChannels = im.shape

print("inputImage: "+inputImage)
print("dimensions: "+str(inputWidth)+"x"+str(inputHeight))

# loop principal
for step in range(len(arrFrames)-initialStep):

  stepUsar = step + initialStep
  frameFilename = finalOutputDir+"/"+ ('%06d' % stepUsar) + "." + inputImage[-3:]

  frameZoom = arrFrames[stepUsar][0]
  frameModel = arrFrames[stepUsar][1]
  if frameModel == None:
    frameModel = 0
  frameLetra = arrFrames[stepUsar][2]
  frameLetraColor = arrFrames[stepUsar][3]
  frameLetraSize = arrFrames[stepUsar][4]
  frameLetraPosX = arrFrames[stepUsar][5]
  frameLetraPosY = arrFrames[stepUsar][6]
  frameHueRotationBool = arrFrames[stepUsar][7]
  # impede repetir letra quando não está fazendo hue rotation
  if frameHueRotationBool != True:
    if frameLetra == arrFrames[stepUsar-1][2]:
      frameLetra = None

  cleanAllDirs()

  # Neural upscale 4x
  !cp "{inputImage}" "{dir_input}"
  model_file = getModelByIndex(frameModel)
  !python upscale.py "{model_file}" --input "{dir_input}" --output "{dir_upscaled}"
  upscaledImage = dir_upscaled+os.listdir(dir_upscaled)[0]
  !mv "{upscaledImage}" "{frameFilename}"

  # reduz a imagem para tamanho um pouco maior que original e faz crop para tamanho original
  resizePorcento = 25 + (frameZoom * 2)
  downSampleQuality = 'high'
  if downSampleQuality == 'high':
    !convert "{frameFilename}" -resize "{resizePorcento}%" -gravity Center -crop "{inputWidth}x{inputHeight}+0+0" -background none +repage "{frameFilename}"
  elif downSampleQuality == 'medium':
    !convert "{frameFilename}" -scale "{resizePorcento}%" -gravity Center -crop "{inputWidth}x{inputHeight}+0+0" -background none +repage "{frameFilename}"
  elif downSampleQuality == 'low':
    !convert "{frameFilename}" -interpolate Nearest -filter point -resize "{resizePorcento}%" -gravity Center -crop "{inputWidth}x{inputHeight}+0+0" -background none +repage "{frameFilename}"

  # coloca letra, se tiver
  if frameLetra != "" and frameLetra != None:
    textImage = generateTextImage(frameLetra,frameLetraColor,frameLetraSize,frameLetraPosX,frameLetraPosY,tmpDir,inputWidth,inputHeight,getCurrentHue(arrFrames,stepUsar))
    !convert {frameFilename} {textImage} -gravity center -composite {frameFilename}

  assert os.path.isfile(frameFilename)
  print('File saved as '+frameFilename)
  
  # coloca frame atual como input para o próximo step
  inputImage = frameFilename

  if step % 10 == 9:
    output.clear()

!rm -Rf tmpDir
output.clear()
print("Acabou")

In [ ]:
#@title #Gera e faz download do vídeo
from google.colab import files

imagesDir = "/content/mydrive/clipe/output"
videoSemSom = "/content/mydrive/clipe/video-sem-som.mp4"
finalVideo = "/content/mydrive/clipe/video-com-musica.mp4"
inputFps = 30 
outputFps =  30
mp3File = '/content/mydrive/sobreumamesa-final.mp3'

assert os.path.isdir(imagesDir)
assert os.path.isfile(mp3File)

%cd {imagesDir}

# gera video
print("Gerando video...")
!ffmpeg -hide_banner -loglevel error -y -r {inputFps} -i %06d.png -c:v libx264 -vf fps={outputFps} -pix_fmt yuv420p {videoSemSom}
# coloca música no vídeo
if os.path.isfile(mp3File):
  !ffmpeg -hide_banner -loglevel error -i {videoSemSom} -i {mp3File} -map 0:v -map 1:a -c:v copy -shortest {finalVideo}